# Análise de Logs do Painel de Sensores de Incêndio

Este notebook processa os logs do painel de sensores de incêndio e extrai informações relevantes para análise.

In [1]:
import pandas as pd
import re
import datetime
import plotly.express as px

In [2]:
def ler_arquivo_log(caminho):
    """Lê o arquivo de log e retorna seu conteúdo"""
    with open(caminho, 'r', encoding='utf-8') as file:
        return file.read()

In [3]:
def extrair_informacoes(conteudo):
    """Extrai informações do conteúdo do log usando regex"""
    # Padrão para encontrar as linhas de dados
    padrao = r'^\s*(\d+)\s+([\w\-\d]+)\s+([*]?[\d\.]+/\d+)\s+(\d+)\s+(\d+/\s*\d+%)\s+(\d+/\s*\d+%)\s+(\w+)'
    
    dados = []
    channel = None
    data_hora = None
    
    for linha in conteudo.split('\n'):
        # Extrair informação do canal
        canal_match = re.search(r'Channel (\d+) \((M\d+)\)', linha)
        if canal_match:
            channel = canal_match.group(1)
            continue
            
        # Extrair data e hora
        data_match = re.search(r'(\d{2}:\d{2}:\d{2})\s+(\w{3})\s+(\d{2})-(\w{3})-(\d{2})', linha)
        if data_match:
            hora = data_match.group(1)
            dia_semana = data_match.group(2)
            dia = data_match.group(3)
            mes = data_match.group(4)
            ano = data_match.group(5)
            data_hora = f"{dia}/{mes}/20{ano} {hora}"
            continue
            
        # Extrair dados dos sensores
        match = re.match(padrao, linha)
        if match:
            dev_num, label, alarm_at, avg_val, current_alarm, peak_alarm, state = match.groups()
            
            dados.append({
                'Channel': channel,
                'DateTime': data_hora,
                'DeviceNumber': dev_num.strip(),
                'Label': label.strip(),
                'AlarmAt': alarm_at.strip(),
                'AverageValue': avg_val.strip(),
                'CurrentAlarm': current_alarm.strip(),
                'PeakAlarm': peak_alarm.strip(),
                'State': state.strip()
            })
    
    return pd.DataFrame(dados)

In [4]:
# Ler e processar o arquivo
caminho_arquivo = 'data/TrueAlarmService.txt'
conteudo = ler_arquivo_log(caminho_arquivo)
df = extrair_informacoes(conteudo)

# Exibir as primeiras linhas do DataFrame
print("Dimensões do DataFrame:", df.shape)
df.head()

Dimensões do DataFrame: (220, 9)


,Channel,DateTime,DeviceNumber,Label,AlarmAt,AverageValue,CurrentAlarm,PeakAlarm,State
0,1,13/FEB/2025 10:11:47,18,N1-L01-018-DF-FRALDARIO-TE-PE,*2.5/131,63,65/ 0%,202/100%,NOR
1,1,13/FEB/2025 10:11:49,56,N1-L01-056-DF-CIRCULACAO-TE-PE,2.5/146,78,79/ 0%,83/ 4%,NOR
2,1,13/FEB/2025 10:11:49,57,N1-L01-057-DF-CIRCULACAO-TE-PE,2.5/130,62,62/ 0%,82/ 28%,NOR
3,1,13/FEB/2025 10:11:49,58,N1-L01-058-DF-CIRCULACAO-TE-PE,*2.5/122,54,54/ 0%,255/100%,NOR
4,1,13/FEB/2025 10:11:49,59,N1-L01-059-DF-CIRCULACAO-TE-PE,*2.5/122,54,53/ 0%,148/100%,NOR


In [ ]:
# Processamento adicional dos dados
def processar_dados(df):
    # Converter DateTime para datetime
    df['DateTime'] = pd.to_datetime(df['DateTime'], format='%d/%b/%Y %H:%M:%S')
    
    # Extrair valores numéricos dos campos de alarme
    df['AlarmThreshold'] = df['AlarmAt'].apply(lambda x: float(x.split('/')[0]))
    df['AlarmValue'] = df['AlarmAt'].apply(lambda x: int(x.split('/')[1]))
    
    # Extrair valores de Current e Peak Alarm
    df['CurrentAlarmValue'] = df['CurrentAlarm'].apply(lambda x: int(x.split('/')[0]))
    df['CurrentAlarmPercentage'] = df['CurrentAlarm'].apply(lambda x: int(x.split('/')[1].replace('%', '').strip()))
    
    df['PeakAlarmValue'] = df['PeakAlarm'].apply(lambda x: int(x.split('/')[0]))
    df['PeakAlarmPercentage'] = df['PeakAlarm'].apply(lambda x: int(x.split('/')[1].replace('%', '').strip()))
    
    # Extrair informações do Label
    df['Location'] = df['Label'].apply(lambda x: x.split('-')[-2] if len(x.split('-')) > 2 else '')
    df['SensorType'] = df['Label'].apply(lambda x: x.split('-')[-1] if len(x.split('-')) > 1 else '')
    
    return df

df_processado = processar_dados(df)
df_processado.head()

### Análises Básicas

In [ ]:
# Estatísticas por Canal
print("\nDistribuição de sensores por canal:")
print(df_processado['Channel'].value_counts())

# Estatísticas de alarmes
print("\nEstatísticas de valores de alarme:")
print(df_processado[['AlarmValue', 'CurrentAlarmValue', 'PeakAlarmValue']].describe())

# Contagem de estados
print("\nDistribuição de estados:")
print(df_processado['State'].value_counts())

In [ ]:
# Visualização da distribuição de valores de alarme
fig = px.box(df_processado, 
             y=['AlarmValue', 'CurrentAlarmValue', 'PeakAlarmValue'],
             title='Distribuição dos Valores de Alarme')
fig.show()

# Visualização por localização
fig = px.bar(df_processado['Location'].value_counts().reset_index(),
            x='index',
            y='Location',
            title='Quantidade de Sensores por Localização')
fig.show()

### Identificação de Sensores Críticos

In [ ]:
# Identificar sensores com altos valores de pico
sensores_criticos = df_processado[df_processado['PeakAlarmPercentage'] > 80].sort_values('PeakAlarmPercentage', ascending=False)

print("Sensores com valores de pico críticos (>80%):")
sensores_criticos[['DeviceNumber', 'Label', 'PeakAlarmValue', 'PeakAlarmPercentage', 'Location']].head(10)

### analises

In [6]:
df.head(15)

,Channel,DateTime,DeviceNumber,Label,AlarmAt,AverageValue,CurrentAlarm,PeakAlarm,State
0,1,2025-02-13 10:11:47,18,N1-L01-018-DF-FRALDARIO-TE-PE,*2.5/131,63,65/ 0%,202/100%,NOR
1,1,2025-02-13 10:11:49,56,N1-L01-056-DF-CIRCULACAO-TE-PE,2.5/146,78,79/ 0%,83/ 4%,NOR
2,1,2025-02-13 10:11:49,57,N1-L01-057-DF-CIRCULACAO-TE-PE,2.5/130,62,62/ 0%,82/ 28%,NOR
3,1,2025-02-13 10:11:49,58,N1-L01-058-DF-CIRCULACAO-TE-PE,*2.5/122,54,54/ 0%,255/100%,NOR
4,1,2025-02-13 10:11:49,59,N1-L01-059-DF-CIRCULACAO-TE-PE,*2.5/122,54,53/ 0%,148/100%,NOR
5,1,2025-02-13 10:11:49,60,N1-L01-060-DF-CIRCULACAO-TE-PE,2.5/146,78,78/ 0%,79/ 0%,NOR
6,1,2025-02-13 10:11:50,101,N1-L01-101-DF-BHS001-TE-PE,2.5/136,68,68/ 0%,69/ 0%,NOR
7,1,2025-02-13 10:11:50,102,N1-L01-102-DF-BHS001-TE-PE,2.5/135,67,67/ 0%,67/ 0%,NOR
8,1,2025-02-13 10:11:50,103,N1-L01-103-DF-BHS001-TE-PE,2.5/136,68,68/ 0%,69/ 0%,NOR
9,1,2025-02-13 10:11:50,105,N1-L01-105-DF-BHS001-TE-PE,2.5/136,68,68/ 0%,69/ 0%,NOR


In [12]:
#df =df[["State"]=="NOR"]
df.State.value_counts()

NOR    220
Name: State, dtype: int64